In [ ]:
# データの永続化
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

In [ ]:
# ソースからのHuggingface Transformersのインストール
!git clone https://github.com/huggingface/transformers -b v4.4.2
!pip install -e transformers
!pip install fugashi[unidic-lite]
!pip install ipadic

In [ ]:
# メニュー「ランタイム → ランタイムを再起動」で「Google Colab」を再起動

# 作業フォルダに移動
%cd '/content/drive/My Drive/work/'

In [ ]:
# Huggingface Datasetsのインストール
!pip install datasets==1.2.1

In [ ]:
# 依存パッケージのインストール
!pip install rouge_score==0.0.4
!pip install sentencepiece==0.1.91

In [ ]:
%%time

# ファインチューニングの実行
!python ./transformers/examples/seq2seq/run_summarization.py \
    --model_name_or_path=sonoisa/t5-base-japanese \
    --do_train \
    --do_eval \
    --train_file=train.csv \
    --validation_file=dev.csv \
    --num_train_epochs=10 \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --output_dir=output/ \
    --predict_with_generate \
    --use_fast_tokenizer=False \
    --logging_steps=100

In [ ]:
# 学習状況の確認
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained('sonoisa/t5-base-japanese') 
model = AutoModelForSeq2SeqLM.from_pretrained('output/')    

# テキスト
text = "おばあさんが川でせんたくをしていると、ドンブラコ、ドンブラコと、大きな桃が流れてきました。おばあさんは大きな桃をひろいあげて、家に持ち帰りました。そして、おじいさんとおばあさんが桃を食べようと桃を切ってみると、なんと中から元気 の良い男の赤ちゃんが飛び出してきました。"

# テキストをテンソルに変換
input = tokenizer.encode(text, return_tensors='pt', max_length=512, truncation=True)

# 推論
model.eval()
with torch.no_grad():
    summary_ids = model.generate(input)
    print(tokenizer.decode(summary_ids[0]))

In [ ]:
%%time

# CNN/DailyMailデータセットによるファインチューニングの実行
!python ./transformers/examples/seq2seq/run_summarization.py \
    --model_name_or_path=t5-small \
    --do_train \
    --do_eval \
    --dataset_name=cnn_dailymail \
    --dataset_config="3.0.0" \
    --source_prefix="summarize: " \
    --num_train_epochs=1 \
    --per_device_train_batch_size=12 \
    --per_device_eval_batch_size=12 \
    --output_dir=output/ \
    --overwrite_output_dir \
    --predict_with_generate